# Weekly Database
Routine for pulling data from MFL and getting it ready for Tableau.

## Init routines

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import MFL

In [2]:
# Create manual lists
pos_order = ['QB', 'RB', 'WR', 'TE', 'PK']
pos_pal = [sns.xkcd_rgb["blurple"],
           sns.xkcd_rgb["blood red"], 
           sns.xkcd_rgb["swamp"], 
           sns.xkcd_rgb["bile"],  
           sns.xkcd_rgb["radioactive green"]]

pal1 = sns.cubehelix_palette(6, rot=-.25, light=.7)
pal1 = pal1[0:5]
pal1 = pal1[::-1]

## ETL
Download, clean, and parse:
- player data
- player scores
- weekly matchup results

### Download data

In [3]:
# Download player data
players = MFL.GetPlayers(season = '2018')
players.head()

,player_id,name,position,status,team
0,0151,"Bills, Buffalo",TMWR,NaN,BUF
1,0152,"Colts, Indianapolis",TMWR,NaN,IND
2,0153,"Dolphins, Miami",TMWR,NaN,MIA
3,0154,"Patriots, New England",TMWR,NaN,NEP
4,0155,"Jets, New York",TMWR,NaN,NYJ


In [4]:
# Download player scores
league_id = '27378'
season = '2018'
weeks = list(range(1, 18))
json = '1'

for w in weeks:
    url = ('http://www71.myfantasyleague.com/'
           + season 
           + '/export?TYPE=playerScores&L='
           + league_id
           + '&W='
           + str(w)
           + '&YEAR='
           + season
           + '&PLAYERS=&POSITION=&STATUS=&RULES=&COUNT=&JSON=' 
           + str(json))
    
    data = pd.read_json(url)
    if w == 1:
        scores = data.playerScores.playerScore
        scores = pd.DataFrame.from_dict(scores, orient = 'columns')
        scores['week'] = w
    else:
        s = data.playerScores.playerScore
        s = pd.DataFrame.from_dict(s, orient = 'columns')
        s['week'] = w
        scores = pd.concat([scores, s])
scores['season'] = season
scores.drop(columns = 'isAvailable', inplace = True)
scores.rename(columns = {'id' : 'player_id', 'score' : 'points'}, inplace = True)
scores['player_id'] = scores['player_id'].astype('str')
scores.head()

,player_id,points,week,season
0,8062,50.3,1,2018
1,4925,37.6,1,2018
2,13116,36.3,1,2018
3,12801,35.3,1,2018
4,13132,34.1,1,2018


In [5]:
# Download weekly matchup results
league_id = '27378'
season = '2018'
weeks = list(range(1, 18))
json = '1'
# Sample url:
#   http://www71.myfantasyleague.com/2018/export?TYPE=weeklyResults&L=27378&APIKEY=axNv2ciQvuKnx1%2BmOl7CZjoeEbox&W=1&JSON=1

results = pd.DataFrame()
# iterate through weeks
for w in weeks:
    #print('Week ' + str(w))
    url = ('http://www71.myfantasyleague.com/'
           + season 
           + '/export?TYPE=weeklyResults&L='
           + league_id
           + '&APIKEY=&W='
           + str(w)
           + '&JSON=' 
           + str(json))
    #print(url)
    data = pd.read_json(url)
    data = data.weeklyResults
    
    # Find number of matchups for the week and use it to limit the iterator. There are normally
    #   5 matchups per week but the data doesnt reflect that so.
    try:
        mlen = len(data['matchup'])
    except:
        print('Week ' + str(w) + ' No Matchups')
        continue
        
    matchups = list(range(mlen))
    # iterate through matchups
    if w == 1:
        for m in matchups:
            r = pd.DataFrame.from_dict(data['matchup'][m]['franchise'], orient = 'columns')
            r['game'] = m + 1
            r['week'] = w
            if m == 0:
                results = r
            else:
                results = pd.concat([results, r])
            #print(' m = ' + str(m))
        
    else:
        for m in matchups:
            r = pd.DataFrame.from_dict(data['matchup'][m]['franchise'], orient = 'columns')
            r['game'] = m + 1
            r['week'] = w
            results = pd.concat([results, r])
            #print(' m = ' + str(m))
            
results['season'] = season
results = results[['season', 'week', 'game', 'id', 'isHome', 'starters', 'nonstarters', 'optimal', 'score', 'result', 'opt_pts', 'player']]
results = results.rename(columns = {'id' : 'owner_id', 'score' : 'team_score' })
results.head()

C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Week 17 No Matchups


,season,week,game,owner_id,isHome,starters,nonstarters,optimal,team_score,result,opt_pts,player
0,2018,1,1,0007,0,"7813,13130,8658,9988,10983,12652,13189,6789,0522,","13617,12152,11668,13116,","13116,11668,12652,0522,6789,13189,8658,9988,12...",105.9,W,143.7,"[{'status': 'starter', 'id': '7813', 'shouldSt..."
1,2018,1,1,0001,1,"10700,13138,8670,12181,12186,9902,11244,12956,...","11248,13607,12930,0527,8360,","10700,9902,0527,12186,8670,12956,11248,8360,12...",75.7,L,110.3,"[{'status': 'starter', 'id': '10700', 'shouldS..."
0,2018,1,2,0008,0,"4925,12634,12171,12205,11232,9884,11250,10976,...","13188,9427,9898,7877,","4925,12171,9884,10976,0518,13188,12634,9427,11...",109.9,L,116.6,"[{'status': 'starter', 'id': '4925', 'shouldSt..."
1,2018,1,2,0005,1,"7394,12151,13132,10527,11671,11938,10312,9686,...","9918,12141,11239,13164,","13132,7394,11671,0528,9686,10312,9918,12151,13...",151.7,W,173.8,"[{'status': 'starter', 'id': '7394', 'shouldSt..."
0,2018,1,3,0003,0,"5848,11660,10729,11705,13604,10738,9831,8359,0...","10389,12175,13192,12157,","5848,13604,0511,9831,8359,11705,10389,10729,10...",105.5,L,117.5,"[{'status': 'starter', 'id': '5848', 'shouldSt..."


### Blend data
Combine the three datasets into a single tidy dataset for Tableau.

In [6]:
# Parse out the starters and non-starters from the weekly results
# initialize objects
row_data = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id'])
starters = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'player_id', 'starter'])
nonstarters = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'player_id', 'starter'])

row_data = pd.DataFrame.from_dict(data = {'season' : results['season'].tolist(),
                                          'week' : results['week'].tolist(),
                                          'game' : results['game'].tolist(),
                                          'owner_id' : results['owner_id'].tolist()})

starter_lists = results['starters'].tolist()
nonstarter_lists = results['nonstarters'].tolist()

# create dataframes of starters and nonstarters for each row
rows = range(len(results))
for r in rows:
    #print('row: ' + str(r))
    season = row_data['season'][r]
    week = row_data['week'][r]
    game = row_data['game'][r]
    owner_id = row_data['owner_id'][r]
    
    # assign starters
    row_starters = starter_lists[r][0:-1]
    slist = row_starters.split(',')
    pdf = pd.DataFrame.from_dict(data = {'season' : season,
                                         'week' : week,
                                         'game' : game,
                                         'owner_id' : owner_id,
                                         'player_id' : slist,
                                         'starter' : True})
    starters = pd.concat([starters, pdf])
    
    # assign nonstarters
    row_nonstarters = nonstarter_lists[r][0:-1]
    slist = row_nonstarters.split(',')
    pdf = pd.DataFrame.from_dict(data = {'season' : season,
                                         'week' : week,
                                         'game' : game,
                                         'owner_id' : owner_id,
                                         'player_id' : slist,
                                         'starter' : False})
    nonstarters = pd.concat([nonstarters, pdf])
    
# merge starters and nonstarters
starter_data = pd.concat([starters, nonstarters])
starter_data['week'] = starter_data['week'].astype('int64')

In [7]:
# Combine score, starter data, and player data
detailed_scores = pd.merge(scores, starter_data, 
                           how = 'left', 
                           left_on = ['player_id', 'season', 'week'], 
                           right_on = ['player_id', 'season', 'week'])
print(len(detailed_scores))

detailed_scores.owner_id.fillna(value = '0000', inplace = True)
detailed_scores.game.fillna(value = 0, inplace = True)
detailed_scores.starter.fillna(value = False, inplace = True)

detailed_scores = pd.merge(detailed_scores, players, 
                           how = 'left', 
                           left_on = 'player_id', 
                           right_on = 'player_id')

detailed_scores = detailed_scores[['season', 
                                   'week', 
                                   'game', 
                                   'player_id', 
                                   'points', 
                                   'name', 
                                   'position', 
                                   'team', 
                                   'owner_id', 
                                   'starter']]

detailed_scores.head()

7545


,season,week,game,player_id,points,name,position,team,owner_id,starter
0,2018,1,5,8062,50.3,"Fitzpatrick, Ryan",QB,TBB,0010,False
1,2018,1,2,4925,37.6,"Brees, Drew",QB,NOS,0008,True
2,2018,1,1,13116,36.3,"Mahomes, Patrick",QB,KCC,0007,False
3,2018,1,3,12801,35.3,"Hill, Tyreek",WR,KCC,0009,True
4,2018,1,2,13132,34.1,"Kamara, Alvin",RB,NOS,0005,True


In [8]:
detailed_scores.to_csv('detailed_scores.csv')